In [19]:
#This is magic, don't touch it
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from html import unescape
import pandas as pd


In [12]:
with open('../corpus/registroactocom.csv', 'r') as file:
    unescaped_text = unescape(file.read())
    with open('../corpus/unescaped_text.csv', 'w') as unescaped_file:
        unescaped_file.write(unescaped_text)
    with open('../corpus/clean_text.csv', 'w') as clean_file:    
        clean_file.write(unescaped_text.replace('<p>', '')\
                         .replace('</p>', '')\
                         .replace('\r','')\
                         .replace('\xa0', ' '))
    

In [14]:
raw_dataset = pd.read_csv('../corpus/clean_text.csv', header=0)

In [15]:
raw_dataset.head()

,segmento_id,acto_id,titulo,palabrasclave,transcripcion,latitudp,longitudp
0,1,1,Presentación,NaN,DEMETRIA: Nada más que yo orita ando previnien...,19.449715,-101.731987
1,2,2,NaN,NaN,"MANUEL BAUTISTA: nosotros analizamos co, cómo,...",19.670720,-101.554051
2,3,5,Oficio de familia,Historia de vida,"00:00\nBERENICE: ¿Nos puede decir su nombre, s...",19.450582,-101.733160
3,4,4,Presentación,NaN,"VIDAL: […] Y, este… este año fue que , que hiz...",19.696462,-101.198387
4,499,22,Nuevo Amanecer,NaN,SANTIAGO: Y tampoco queremos este obligarlos a...,0.000000,0.000000


In [16]:
new_dataset = raw_dataset.copy()

In [17]:
new_dataset.drop(columns=['titulo','palabrasclave','acto_id','latitudp','longitudp'])

,segmento_id,transcripcion
0,1,DEMETRIA: Nada más que yo orita ando previnien...
1,2,"MANUEL BAUTISTA: nosotros analizamos co, cómo,..."
2,3,"00:00\nBERENICE: ¿Nos puede decir su nombre, s..."
3,4,"VIDAL: […] Y, este… este año fue que , que hiz..."
4,499,SANTIAGO: Y tampoco queremos este obligarlos a...
...,...,...
404,659,GEORGINA: ¿Y sí encontraron ellos el dinero?\n...
405,660,"GEORGINA: ¿O sea no a cualquiera les toca, pue..."
406,661,GEORGINA: ¿Su papá entonces tampoco agarro din...
407,662,GEORGINA: ¿Cómo hace cuanto dejo de escuchar q...


In [18]:
new_dataset.transcripcion[0]


'DEMETRIA: Nada más que yo orita ando previniendo para bañiar, [ah] eso es que lo andaba haciendo ya, sí.\nBERENICE: A ver, no nos, no nos tardamos mucho.\nDEMETRIA: Ajá, sí. \nBERENICE: ¿Quién va a preguntar?\nGEORGINA: Pues, si quiere empezamos con su nombre [mjm] ¿Cómo se llama usted?\nDEMETRIA: Yo me llamo Demetria.\nGEORGINA: Demetria.\nBERENICE: ¿Y cómo se apellida? ¿Demetria qué?\nDEMETRIA: Antonio Guzmán [mjm]. \nBERENICE: ¿Y cuántos?\nGEORGINA: ¿Y cuántos años tiene? \nDEMETRIA: No, pos ya tengo setenta, setenta y dos.\nITZI: ¿Y nació aquí en la Pacanda? \nDEMETRIA: Sí, aquí, somos de aquí pues, nativos, sí.\nGEORGINA: ¿Su esposo también?\nDEMETRIA: Sí, también él, también sí, también ya está grande pues también él. Él lleva más, ganó, este, 4 años, me gana a mí. [Oh] Sí, sí. \n '

In [20]:
new_dataset.to_csv('../corpus/new_dataset.csv')

In [21]:
corpus = new_dataset.transcripcion.astype('str')

In [23]:
tfidf_vectorizer = TfidfVectorizer()

In [24]:
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

In [27]:
print(tfidf_matrix)

  (0, 5838)	0.07683599486416617
  (0, 5568)	0.05220013136239172
  (0, 3920)	0.11614661436913616
  (0, 3591)	0.02315241145816585
  (0, 3930)	0.1034142766775673
  (0, 4975)	0.07128343366745336
  (0, 4003)	0.04793245304619705
  (0, 3632)	0.029971805987293394
  (0, 9042)	0.10978767418658493
  (0, 7971)	0.1365476510726575
  (0, 3547)	0.07295448338532443
  (0, 5622)	0.1086986742730498
  (0, 2553)	0.01947688746191057
  (0, 7794)	0.06580045164653484
  (0, 5970)	0.05220013136239172
  (0, 4756)	0.01966823169675617
  (0, 3182)	0.020911526188785598
  (0, 824)	0.06980711898696919
  (0, 5591)	0.08851839648539458
  (0, 4583)	0.08658304176948912
  (0, 2992)	0.04156082214837933
  (0, 7651)	0.17316608353897825
  (0, 8095)	0.051643456020621106
  (0, 6560)	0.04727242226451916
  (0, 8157)	0.03891111832905225
  :	:
  (407, 2065)	0.02210042811574396
  (407, 7668)	0.09008843263389638
  (407, 6800)	0.09871952665804555
  (407, 3957)	0.0511863522069947
  (407, 5500)	0.08737620120411807
  (407, 5683)	0.1247217236

In [28]:
for i, feature in enumerate(tfidf_vectorizer.get_feature_names()):
    print(i, feature)

0 00
1 000
2 000000
3 0001pt
4 000a
5 007b62b209a7
6 01
7 03
8 05
9 06
10 07
11 08c0590b088e
12 0be5
13 0cm
14 0pt
15 10
16 1050
17 107
18 11
19 115
20 1177
21 11pt
22 12
23 1250
24 12a6
25 12pt
26 13
27 14
28 1440
29 1450
30 149
31 1495
32 15
33 150
34 16
35 163
36 168
37 16px
38 18
39 1850
40 19
41 1927
42 1936
43 1945
44 1953
45 1960
46 1965
47 1972
48 1977
49 1ca510489c08
50 20
51 200
52 2000
53 2004
54 2005
55 2007
56 2008
57 2017
58 21
59 22
60 23
61 25
62 25pt
63 26
64 28
65 29
66 29f5
67 30
68 31
69 32
70 33
71 34
72 35
73 350
74 36
75 36bb
76 36pt
77 37
78 375
79 39
80 39af
81 3ad5
82 3d
83 3d1c74ed
84 3da8752c
85 40
86 400
87 41
88 42
89 43
90 44
91 45
92 450
93 45b65e14669a
94 46
95 4664ce00
96 47
97 48
98 49
99 4d80bbf58351
100 4d979725
101 4pt
102 50
103 51
104 52
105 53
106 55
107 56
108 57
109 5720662472ea
110 59eee6453c87
111 5ad67ddae1aa
112 5f5b
113 5pt
114 60
115 600
116 61
117 62
118 62fb
119 63
120 64
121 65
122 66
123 67
124 68
125 683b
126 69
127 6pt
128 70
129 7

1172 bañiar
1173 baño
1174 baños
1175 baúl
1176 bda03e596c9b
1177 be
1178 beber
1179 bebé
1180 bebés
1181 beca
1182 becerrito
1183 becerro
1184 before
1185 bendecido
1186 bendice
1187 bendicieron
1188 bendiga
1189 bendijeron
1190 bendita
1191 benéfico
1192 bere
1193 berenice
1194 besarle
1195 beso
1196 beto
1197 biblia
1198 bibliography
1199 biblioteca
1200 bidi
1201 bie
1202 bien
1203 bienes
1204 bienvenidos
1205 biera
1206 bigote
1207 bigotón
1208 bila
1209 bisabuelo
1210 bisabuelos
1211 bistec
1212 bisteces
1213 bióloga
1214 biólogo
1215 biólogos
1216 black
1217 blanca
1218 blancas
1219 blanco
1220 blancos
1221 blandas
1222 blanquito
1223 block
1224 blusa
1225 blusas
1226 bo
1227 boca
1228 bocina
1229 boda
1230 bodas
1231 body
1232 bola
1233 bolas
1234 bolsa
1235 bolsas
1236 bomba
1237 bombas
1238 bombean
1239 bondad
1240 bonifacio
1241 bonita
1242 bonitas
1243 bonito
1244 bonitos
1245 book
1246 boquita
1247 borachilla
1248 borbujito
1249 bordar
1250 borde
1251 border
1252 borra
125

2288 costales
2289 costalito
2290 costillitas
2291 costitante
2292 costoso
2293 costosos
2294 costumbre
2295 costumbres
2296 costura
2297 count
2298 coyote
2299 coyotes
2300 cre
2301 crea
2302 creando
2303 creas
2304 crece
2305 crecer
2306 crecido
2307 creciendo
2308 creciera
2309 crecieron
2310 crecimos
2311 creció
2312 crecí
2313 crecía
2314 crecían
2315 cree
2316 creemos
2317 creen
2318 creencia
2319 creer
2320 crees
2321 crefal
2322 crema
2323 crenulan
2324 creo
2325 creoque
2326 crepé
2327 creyó
2328 creé
2329 creía
2330 creíamos
2331 creímos
2332 creó
2333 criaban
2334 criadero
2335 criar
2336 criatura
2337 crie
2338 criollo
2339 cristalina
2340 cristalino
2341 cristera
2342 cristeros
2343 cristiana
2344 cristo
2345 cruces
2346 crucita
2347 cruz
2348 cruzaba
2349 cruzando
2350 cruzar
2351 cruzarla
2352 cruzó
2353 cráneo
2354 cría
2355 crías
2356 críticas
2357 cu
2358 cua
2359 cuadra
2360 cuadrados
2361 cuadras
2362 cuadrito
2363 cuadro
2364 cuajado
2365 cual
2366 cuales
2367 cual

3262 engañarlos
3263 engañarme
3264 engaño
3265 engrandeció
3266 enjambre
3267 enjaulado
3268 enka
3269 enkari
3270 enoja
3271 enojado
3272 enojar
3273 enojas
3274 enorgullecido
3275 enorme
3276 enormes
3277 enreda
3278 enredó
3279 enrique
3280 enriquecer
3281 ensalada
3282 ensallar
3283 ensartaran
3284 ensayaban
3285 ensayan
3286 ensayando
3287 ensayar
3288 ensayen
3289 enseguida
3290 enseñ
3291 enseña
3292 enseñaban
3293 enseñado
3294 enseñamos
3295 enseñando
3296 enseñanza
3297 enseñar
3298 enseñarle
3299 enseñarles
3300 enseñarlo
3301 enseñarme
3302 enseñaron
3303 enseñas
3304 enseñe
3305 enseño
3306 enseñábamos
3307 enseñé
3308 enseñó
3309 ensuciemos
3310 enséñame
3311 enséñate
3312 entendemos
3313 entender
3314 entenderlo
3315 entendía
3316 entendían
3317 enterado
3318 enteramos
3319 enteraron
3320 enterarse
3321 enterraba
3322 enterraban
3323 enterrada
3324 enterradas
3325 enterrado
3326 enterrar
3327 enterrarlo
3328 enterrarlos
3329 enterraron
3330 entiendas
3331 entiende
3332 

4582 itsï
4583 itzi
4584 ixhuatepec
4585 ixu
4586 izquierda
4587 ja
4588 jabón
4589 jacinto
4590 jacob
4591 jacobo
4592 jaja
4593 jajaja
4594 jajtaku
4595 jajája
4596 jaka
4597 jala
4598 jalado
4599 jalamos
4600 jalando
4601 jalar
4602 jalaron
4603 jalisco
4604 jaló
4605 jamás
4606 jani
4607 janitzio
4608 janitzios
4609 janiztio
4610 japi
4611 japingua
4612 japoneses
4613 japón
4614 jara
4615 jarabe
4616 jardines
4617 jardín
4618 jarhani
4619 jaripeo
4620 jaripeos
4621 jarrito
4622 jarrón
4623 jarácuaro
4624 jatsiperi
4625 jatsisti
4626 jauatia
4627 jaula
4628 jaulas
4629 je
4630 jean
4631 jefatura
4632 jefe
4633 jefes
4634 jeje
4635 jerónimo
4636 jessica
4637 jesucristo
4638 jesuita
4639 jesús
4640 ji
4641 jicarita
4642 jicaritas
4643 jijo
4644 jilgueros
4645 jimpo
4646 jinetes
4647 jinkoni
4648 jit
4649 jm
4650 jo
4651 jonto
4652 jorge
4653 jornalero
4654 josefinita
4655 josé
4656 joven
4657 jovenciata
4658 jovencita
4659 jovencitas
4660 jovencito
4661 joya
4662 juan
4663 juana
4664 

5543 musical
5544 musicos
5545 mutuamente
5546 muuy
5547 muy
5548 muéveme
5549 muñecas
5550 muñeco
5551 mx
5552 my
5553 mándalo
5554 mánde
5555 máquina
5556 máquinas
5557 márquez
5558 más
5559 máscaras
5560 mátame
5561 máxima
5562 máximo
5563 médico
5564 médicos
5565 méndez
5566 mérito
5567 méxico
5568 mí
5569 mía
5570 mínimamente
5571 mío
5572 mírame
5573 música
5574 músico
5575 músicos
5576 na
5577 naada
5578 nacatamales
5579 nace
5580 nacida
5581 nacido
5582 nacidos
5583 naciendo
5584 naciera
5585 nacieron
5586 nacimiento
5587 nacimos
5588 nacional
5589 nacionalidad
5590 naciste
5591 nació
5592 nación
5593 nací
5594 nacía
5595 nada
5596 nadamás
5597 nadando
5598 nadar
5599 nadie
5600 nadien
5601 nagua
5602 naguas
5603 nahua
5604 nahuas
5605 nailon
5606 namas
5607 name
5608 namá
5609 namás
5610 nan
5611 nana
5612 nani
5613 napizaro
5614 napíz
5615 napízaro
5616 naranja
5617 nari
5618 nariz
5619 narrar
5620 narylim
5621 nativo
5622 nativos
5623 nats
5624 natural
5625 naturales
5626 na

6721 programas
6722 prohibido
6723 prohíben
6724 promesa
6725 prometen
6726 prometido
6727 prometimos
6728 prometió
6729 prometí
6730 prometía
6731 promocional
6732 promueva
6733 pronto
6734 pronunciación
6735 pronunciar
6736 propia
6737 propias
6738 propiedad
6739 propiedades
6740 propietario
6741 propio
6742 propios
6743 proponer
6744 propuso
6745 propósito
6746 prosperidad
6747 protección
6748 protege
6749 proteger
6750 protegerse
6751 proteges
6752 provecho
6753 provengo
6754 provisionalmente
6755 provocamos
6756 proyecto
6757 proyectos
6758 prueba
6759 pruebas
6760 práctica
6761 préstame
6762 préstenme
6763 príncipe
6764 príncpes
6765 prójimo
6766 próxima
6767 próximamente
6768 próximo
6769 ps
6770 pu
6771 puacuaro
6772 puantani
6773 pude
6774 pudiera
6775 pudieran
6776 pudieron
6777 pudimos
6778 pudo
6779 pue
6780 pueblito
6781 pueblitos
6782 pueblo
6783 pueblos
6784 pueda
6785 puedan
6786 puede
6787 pueden
6788 puedes
6789 puedo
6790 puees
6791 puente
6792 puer
6793 puerco
6794 

7873 suciedad
7874 sue
7875 suegra
7876 suegro
7877 suegros
7878 suel
7879 sueldo
7880 suele
7881 suelen
7882 suelo
7883 sueltas
7884 suelte
7885 suelto
7886 suena
7887 suerte
7888 sueña
7889 sueñan
7890 sueño
7891 sueños
7892 suficiente
7893 sufren
7894 sufrimos
7895 sufrir
7896 sufrió
7897 sufro
7898 sugirió
7899 sumamente
7900 sumbans
7901 sumbas
7902 sumen
7903 sumergieron
7904 sumiendo
7905 sumisas
7906 sumió
7907 supe
7908 super
7909 superaba
7910 superior
7911 supieran
7912 supieron
7913 supimos
7914 suplente
7915 supo
7916 suponer
7917 supongo
7918 suponiendo
7919 supportlists
7920 supuestamente
7921 sur
7922 surge
7923 surgen
7924 surgió
7925 surtir
7926 surtía
7927 surtían
7928 sus
7929 suspira
7930 sustentaba
7931 susto
7932 sustraer
7933 suéter
7934 sva
7935 sába
7936 sábado
7937 sácalo
7938 sámano
7939 sánchez
7940 sé
7941 séptica
7942 sépticas
7943 sí
7944 sígale
7945 síguele
7946 sígueme
7947 símbolos
7948 sólida
7949 sólo
7950 súper
7951 ta
7952 tab
7953 taba
7954 taban

8745 verdecito
8746 verdura
8747 verdurita
8748 verduritas
8749 verdá
8750 veredita
8751 verenda
8752 vergüenza
8753 verla
8754 verlas
8755 verle
8756 verlo
8757 verlos
8758 verme
8759 vernos
8760 versiones
8761 versión
8762 versátiles
8763 vertical
8764 verá
8765 verán
8766 verás
8767 ves
8768 vestida
8769 vestidita
8770 vestidito
8771 vestido
8772 vestimenta
8773 vestimentas
8774 vestimos
8775 vestir
8776 vestuario
8777 vestía
8778 vete
8779 vez
8780 veía
8781 veíamos
8782 veían
8783 veías
8784 vi
8785 viacrucis
8786 viacruz
8787 viaj
8788 viajado
8789 viaje
8790 viajes
8791 vicio
8792 vida
8793 vidal
8794 vidas
8795 vide
8796 video
8797 videoclips
8798 videos
8799 vidrio
8800 vieja
8801 viejecita
8802 viejillo
8803 viejita
8804 viejitas
8805 viejito
8806 viejitos
8807 viejo
8808 viendo
8809 viene
8810 vienen
8811 vienes
8812 vieniera
8813 viento
8814 viera
8815 vieran
8816 vieras
8817 viernes
8818 vieron
8819 view
8820 vigilancia
8821 vigilantes
8822 vigilia
8823 vihuela
8824 vimos


In [31]:
tf_vectorizer = CountVectorizer()

In [32]:
tf_matrix = tf_vectorizer.fit_transform(corpus)

In [33]:
for i, feature in enumerate(tf_vectorizer.get_feature_names()):
    print(i, feature)

0 00
1 000
2 000000
3 0001pt
4 000a
5 007b62b209a7
6 01
7 03
8 05
9 06
10 07
11 08c0590b088e
12 0be5
13 0cm
14 0pt
15 10
16 1050
17 107
18 11
19 115
20 1177
21 11pt
22 12
23 1250
24 12a6
25 12pt
26 13
27 14
28 1440
29 1450
30 149
31 1495
32 15
33 150
34 16
35 163
36 168
37 16px
38 18
39 1850
40 19
41 1927
42 1936
43 1945
44 1953
45 1960
46 1965
47 1972
48 1977
49 1ca510489c08
50 20
51 200
52 2000
53 2004
54 2005
55 2007
56 2008
57 2017
58 21
59 22
60 23
61 25
62 25pt
63 26
64 28
65 29
66 29f5
67 30
68 31
69 32
70 33
71 34
72 35
73 350
74 36
75 36bb
76 36pt
77 37
78 375
79 39
80 39af
81 3ad5
82 3d
83 3d1c74ed
84 3da8752c
85 40
86 400
87 41
88 42
89 43
90 44
91 45
92 450
93 45b65e14669a
94 46
95 4664ce00
96 47
97 48
98 49
99 4d80bbf58351
100 4d979725
101 4pt
102 50
103 51
104 52
105 53
106 55
107 56
108 57
109 5720662472ea
110 59eee6453c87
111 5ad67ddae1aa
112 5f5b
113 5pt
114 60
115 600
116 61
117 62
118 62fb
119 63
120 64
121 65
122 66
123 67
124 68
125 683b
126 69
127 6pt
128 70
129 7

1161 batea
1162 batidita
1163 batido
1164 bautista
1165 bautizar
1166 bautizaron
1167 bautizo
1168 bautizos
1169 bañar
1170 bañarnos
1171 bañarse
1172 bañiar
1173 baño
1174 baños
1175 baúl
1176 bda03e596c9b
1177 be
1178 beber
1179 bebé
1180 bebés
1181 beca
1182 becerrito
1183 becerro
1184 before
1185 bendecido
1186 bendice
1187 bendicieron
1188 bendiga
1189 bendijeron
1190 bendita
1191 benéfico
1192 bere
1193 berenice
1194 besarle
1195 beso
1196 beto
1197 biblia
1198 bibliography
1199 biblioteca
1200 bidi
1201 bie
1202 bien
1203 bienes
1204 bienvenidos
1205 biera
1206 bigote
1207 bigotón
1208 bila
1209 bisabuelo
1210 bisabuelos
1211 bistec
1212 bisteces
1213 bióloga
1214 biólogo
1215 biólogos
1216 black
1217 blanca
1218 blancas
1219 blanco
1220 blancos
1221 blandas
1222 blanquito
1223 block
1224 blusa
1225 blusas
1226 bo
1227 boca
1228 bocina
1229 boda
1230 bodas
1231 body
1232 bola
1233 bolas
1234 bolsa
1235 bolsas
1236 bomba
1237 bombas
1238 bombean
1239 bondad
1240 bonifacio
1241 bo

2295 costumbres
2296 costura
2297 count
2298 coyote
2299 coyotes
2300 cre
2301 crea
2302 creando
2303 creas
2304 crece
2305 crecer
2306 crecido
2307 creciendo
2308 creciera
2309 crecieron
2310 crecimos
2311 creció
2312 crecí
2313 crecía
2314 crecían
2315 cree
2316 creemos
2317 creen
2318 creencia
2319 creer
2320 crees
2321 crefal
2322 crema
2323 crenulan
2324 creo
2325 creoque
2326 crepé
2327 creyó
2328 creé
2329 creía
2330 creíamos
2331 creímos
2332 creó
2333 criaban
2334 criadero
2335 criar
2336 criatura
2337 crie
2338 criollo
2339 cristalina
2340 cristalino
2341 cristera
2342 cristeros
2343 cristiana
2344 cristo
2345 cruces
2346 crucita
2347 cruz
2348 cruzaba
2349 cruzando
2350 cruzar
2351 cruzarla
2352 cruzó
2353 cráneo
2354 cría
2355 crías
2356 críticas
2357 cu
2358 cua
2359 cuadra
2360 cuadrados
2361 cuadras
2362 cuadrito
2363 cuadro
2364 cuajado
2365 cual
2366 cuales
2367 cualquier
2368 cualquiera
2369 cuanajo
2370 cuando
2371 cuantas
2372 cuanto
2373 cuantos
2374 cuantucha
2375

3492 esos
3493 espacio
3494 espada
3495 espalda
3496 espaldita
3497 espanta
3498 espantaba
3499 espantaban
3500 espantado
3501 espantan
3502 espantaron
3503 espantaste
3504 espantates
3505 espanto
3506 espantos
3507 esparcido
3508 españa
3509 español
3510 españolas
3511 españoles
3512 españoli
3513 españolicha
3514 especia
3515 especial
3516 especiales
3517 especie
3518 especies
3519 especito
3520 específico
3521 espejo
3522 espejos
3523 espera
3524 esperaba
3525 esperaban
3526 esperamos
3527 esperan
3528 esperando
3529 esperandolos
3530 esperanza
3531 esperar
3532 esperarlos
3533 esperarme
3534 esperaron
3535 esperarse
3536 esperas
3537 esperemos
3538 esperimento
3539 espero
3540 esperó
3541 espierar
3542 espina
3543 espinas
3544 espinábamos
3545 esposa
3546 esposas
3547 esposo
3548 esposos
3549 espérame
3550 espérenme
3551 espérenos
3552 espérense
3553 espírito
3554 espíritu
3555 esque
3556 esqueleto
3557 esqueletos
3558 esquina
3559 esquinas
3560 esquinita
3561 esquiva
3562 est
3563

4723 justo
4724 juventud
4725 juzgue
4726 juí
4727 jája
4728 jájaja
4729 jícara
4730 jícaras
4731 jóven
4732 jóvenes
4733 ka
4734 kamapkia
4735 kampanarhu
4736 karate
4737 kauarito
4738 kauaros
4739 keyboard
4740 kilo
4741 kilos
4742 kilómetros
4743 kiosquito
4744 kueramo
4745 kueramu
4746 kuiripos
4747 kurankua
4748 kurankujparani
4749 kurankunichkia
4750 kurankusini
4751 kurankutatia
4752 kurhankuni
4753 kutsu
4754 kínder
4755 l0
4756 la
4757 labor
4758 laboratorio
4759 laboristas
4760 labrada
4761 labradas
4762 labrarlos
4763 lacustre
4764 ladearse
4765 ladito
4766 ladió
4767 lado
4768 lados
4769 ladraba
4770 ladraron
4771 ladridos
4772 lago
4773 lagos
4774 lagua
4775 laguito
4776 laguna
4777 lagunas
4778 lagunita
4779 laja
4780 lamentando
4781 lamina
4782 laminitas
4783 lana
4784 lance
4785 lancha
4786 lanchas
4787 lancher
4788 lancheros
4789 lang
4790 language
4791 lanita
4792 lapio
4793 larga
4794 largas
4795 largo
4796 largos
4797 largote
4798 larguito
4799 las
4800 lastima
4801

5601 nagua
5602 naguas
5603 nahua
5604 nahuas
5605 nailon
5606 namas
5607 name
5608 namá
5609 namás
5610 nan
5611 nana
5612 nani
5613 napizaro
5614 napíz
5615 napízaro
5616 naranja
5617 nari
5618 nariz
5619 narrar
5620 narylim
5621 nativo
5622 nativos
5623 nats
5624 natural
5625 naturales
5626 naturaleza
5627 navegaba
5628 navegaban
5629 navidad
5630 nayarit
5631 necesariamente
5632 necesario
5633 necesidad
5634 necesidades
5635 necesita
5636 necesitaba
5637 necesitadas
5638 necesitado
5639 necesitamos
5640 necesitan
5641 necesitar
5642 necesitaría
5643 necesitas
5644 necesite
5645 necio
5646 negar
5647 negocio
5648 negra
5649 negras
5650 negrito
5651 negro
5652 nemacha
5653 nescafé
5654 nestor
5655 new
5656 ng
5657 ni
5658 nicaragua
5659 nico
5660 nicodemos
5661 nicolás
5662 nieta
5663 nietas
5664 nieto
5665 nietos
5666 nimodo
5667 ninguna
5668 ninguno
5669 ningún
5670 ningúna
5671 nintakaria
5672 nivel
5673 nixtamal
5674 niñ
5675 niña
5676 niñas
5677 niñez
5678 niñito
5679 niñitos
56

6980 ranchería
6981 ranchito
6982 ranchitos
6983 rancho
6984 ranchos
6985 rapidito
6986 rara
6987 raras
6988 raro
6989 raros
6990 rasgos
6991 raso
6992 rateros
6993 ratitito
6994 ratito
6995 rato
6996 rayaba
6997 rayado
6998 rayar
6999 rayas
7000 rayo
7001 rayos
7002 raza
7003 razones
7004 razón
7005 raíz
7006 raúl
7007 re
7008 reaccionar
7009 reaccionaron
7010 reaccionó
7011 real
7012 reales
7013 realicé
7014 realidad
7015 realización
7016 realizamos
7017 realmente
7018 rebanar
7019 rebasamos
7020 rebozo
7021 recabar
7022 receso
7023 recib
7024 recibe
7025 recibido
7026 recibiendo
7027 recibimos
7028 recibir
7029 recibió
7030 recibían
7031 recien
7032 reciente
7033 recientes
7034 recio
7035 recipiente
7036 recién
7037 recogen
7038 recoger
7039 recogerlo
7040 recogiendo
7041 recogieron
7042 recogió
7043 recogí
7044 recogía
7045 recolectan
7046 recolectar
7047 reconocerle
7048 reconocida
7049 reconocidos
7050 reconocimiento
7051 reconocimos
7052 reconoció
7053 reconocí
7054 reconstrucci

8028 tazas
8029 tda
8030 te
8031 teatro
8032 techo
8033 tecnología
8034 tecolote
8035 tecua
8036 tecuas
8037 tecuen
8038 tecuena
8039 tecuenera
8040 tecuenita
8041 teja
8042 tejabancito
8043 tejadito
8044 tejado
8045 tejados
8046 teje
8047 tejer
8048 tejido
8049 tejiendo
8050 tejió
8051 tejía
8052 tejían
8053 tekua
8054 tekuay
8055 tela
8056 telares
8057 tele
8058 teles
8059 televisa
8060 televisión
8061 teléfono
8062 telésforo
8063 tem
8064 tema
8065 temblor
8066 tembló
8067 temperatura
8068 templo
8069 temporada
8070 temporadas
8071 temprana
8072 tempranito
8073 temprano
8074 tempto
8075 ten
8076 tendemos
8077 tender
8078 tendido
8079 tendidos
8080 tendiendo
8081 tendieron
8082 tendió
8083 tendrá
8084 tendría
8085 tendríamos
8086 tendíamos
8087 tene
8088 tenemos
8089 tenencia
8090 tener
8091 tenga
8092 tengamos
8093 tengan
8094 tengas
8095 tengo
8096 tenia
8097 teniaba
8098 tenido
8099 teniendo
8100 teniábamos
8101 tenochtitlan
8102 tentaba
8103 tentarlos
8104 tenébamos
8105 tenía
81

In [34]:
print(tf_matrix)

  (0, 2660)	10
  (0, 5595)	1
  (0, 5558)	2
  (0, 6859)	2
  (0, 8996)	2
  (0, 5931)	1
  (0, 656)	1
  (0, 6668)	1
  (0, 6045)	1
  (0, 1172)	1
  (0, 447)	1
  (0, 3491)	1
  (0, 3414)	1
  (0, 5022)	1
  (0, 641)	1
  (0, 4160)	1
  (0, 8985)	3
  (0, 7943)	8
  (0, 1193)	4
  (0, 8738)	1
  (0, 5683)	3
  (0, 5718)	2
  (0, 8008)	1
  (0, 5500)	1
  (0, 486)	1
  :	:
  (407, 7147)	1
  (407, 7359)	1
  (407, 9006)	1
  (407, 5811)	1
  (407, 6972)	3
  (407, 7216)	1
  (407, 5509)	1
  (407, 1594)	2
  (407, 4177)	1
  (407, 6297)	1
  (407, 5296)	1
  (407, 8431)	1
  (407, 6490)	1
  (408, 5558)	1
  (408, 6859)	1
  (408, 5683)	1
  (408, 5718)	1
  (408, 3957)	2
  (408, 6800)	2
  (408, 527)	1
  (408, 6910)	1
  (408, 5499)	2
  (408, 3994)	2
  (408, 2174)	1
  (408, 7147)	1


In [35]:
tf_matrix

<409x9069 sparse matrix of type '<class 'numpy.int64'>'
	with 66475 stored elements in Compressed Sparse Row format>

In [40]:
pd.DataFrame(tf_matrix.toarray())[486]

0      1
1      2
2      5
3      2
4      0
      ..
404    1
405    0
406    0
407    0
408    0
Name: 486, Length: 409, dtype: int64

http://www.ultravioletanalytics.com/blog/tf-idf-basics-with-pandas-scikit-learn

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html